# 2. Data Preprocessing 

Now that we have visually explored the data and analyzed correlation and summary statistics, we will process the data for modeling and prediction.

There are several things to consider:

* One-hot enconding
* Scaling of numerical variables
* PCA
* Variable importance

We will progess through these, explaining each step in turn.

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt 
import seaborn as sns

file_path=r'C:\Users\Marco\Desktop\TU Dublin\Programming for Big Data - H6018\2nd Assignment\data'
file_name='/credit_card_default.xls'

df = pd.read_excel(file_path+file_name, encoding = "ISO-8859-1")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
#checking duplicates for ID and drop it
df.ID.duplicated().sum()
df.drop(['ID'], axis=1, inplace=True)
#rename columns
df = df.rename(columns = {'default payment next month' : 'def_next_month', 
                          'PAY_0' : 'PAY_1'})
df.def_next_month.value_counts()
#Correct data inconsistencies 
# MARRIAGE = 0 is deleted
df = df.drop(df[df['MARRIAGE']==0].index)
# EDUCATION = 0, 5 and 6 are deleted
df = df.drop(df[df['EDUCATION']==0].index)
df = df.drop(df[df['EDUCATION']==5].index)
df = df.drop(df[df['EDUCATION']==6].index)
#Fixing PAY variables
for att in ['PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']:
  # categories -2,-1 are grouped into a single class -1: pay duty   
  filter = (df[att] == -2) | (df[att] == -1) 
  df.loc[filter, att] = -1
  df[att] = df[att].astype('int64')
  filter = (df[att] >= 0)
  df.loc[filter, att] = df.loc[filter, att] + 1

### One-Hot Encoding
We can create dummy variables for the variables SEX, EDUCATION and MARRIAGE. They are categorical variables with relatively few values.

In [3]:
# Set 'category' type to categorical attributes
for att in ['SEX', 'EDUCATION', 'MARRIAGE']:
  df[att] = df[att].astype('category')

# one-hot encoding
df_encoded=pd.concat([pd.get_dummies(df['SEX'], prefix='SEX'),
                pd.get_dummies(df['EDUCATION'], prefix='EDUCATION'), 
                pd.get_dummies(df['MARRIAGE'], prefix='MARRIAGE'),
                df],axis=1)
# drop original columns
df_encoded.drop(['EDUCATION'],axis=1, inplace=True)
df_encoded.drop(['SEX'],axis=1, inplace=True)
df_encoded.drop(['MARRIAGE'],axis=1, inplace=True)
# print samples
df_encoded.head()

,SEX_1,SEX_2,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3,LIMIT_BAL,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,def_next_month
0,0,1,0,1,0,0,1,0,0,20000,24,3,3,-1,-1,-1,-1,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,0,1,0,1,0,0,0,1,0,120000,26,-1,3,1,1,1,3,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,0,1,0,1,0,0,0,1,0,90000,34,1,1,1,1,1,1,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,0,1,0,1,0,0,1,0,0,50000,37,1,1,1,1,1,1,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,1,0,0,1,0,0,1,0,0,50000,57,-1,1,-1,1,1,1,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0


### Scaling of Numerical Variables

For the numerical variables, we have different scale - i.e., age, PAY_1 - PAY_6 are numbers between 0 and 100, whereas LIMIT_BAL, BILL_AMT and PAY variables are higher (up to 1M). These variables also display multicollinearity. Therefore we will use PCA to analyze the variance explained by linear combinations of these variables. 

As we have such different range of values, we scale the data by dividing each variable by its standard deviation. 

We do not include def_next_month as it is our variable to be predicted.

In [4]:
# drop response variable and the hot encoded variables
df_drop_var = df_encoded.drop(['def_next_month', 'SEX_1','SEX_2','EDUCATION_1','EDUCATION_2','EDUCATION_3','EDUCATION_4',
                               'MARRIAGE_1','MARRIAGE_2','MARRIAGE_3'],axis=1)

df_scaled = df_drop_var/df_drop_var.std()

df_scaled

,LIMIT_BAL,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,0.153912,2.604946,2.240103,2.147167,-0.719863,-0.740329,-0.763742,-0.754710,0.053332,0.043737,0.009968,0.000000,0.000000,0.000000,0.000000,0.029841,0.000000,0.000000,0.000000,0.000000
1,0.923475,2.822025,-0.746701,2.147167,0.719863,0.740329,0.763742,2.264129,0.036554,0.024322,0.038800,0.050969,0.056920,0.054788,0.000000,0.043310,0.056880,0.063649,0.000000,0.113268
2,0.692606,3.690340,0.746701,0.715722,0.719863,0.740329,0.763742,0.754710,0.398513,0.197775,0.196155,0.223237,0.246263,0.261240,0.091621,0.064965,0.056880,0.063649,0.065599,0.283170
3,0.384781,4.015958,0.746701,0.715722,0.719863,0.740329,0.763742,0.754710,0.640450,0.680066,0.713082,0.441053,0.477089,0.496422,0.120713,0.087444,0.068256,0.070014,0.070125,0.056634
4,0.384781,6.186746,-0.746701,0.715722,-0.719863,0.740329,0.763742,0.754710,0.117445,0.079945,0.518417,0.326187,0.315424,0.321422,0.120713,1.588665,0.568799,0.572845,0.045197,0.038454
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,1.693037,4.233037,0.746701,0.715722,0.719863,0.740329,0.763742,0.754710,2.575267,2.718615,3.014370,1.370856,0.514618,0.268482,0.513029,0.866206,0.284570,0.193940,0.327993,0.056634
29996,1.154343,4.667195,-0.746701,-0.715722,-0.719863,-0.740329,0.763742,0.754710,0.022938,0.025774,0.050663,0.139868,0.085503,0.000000,0.110875,0.152712,0.511805,0.008211,0.000000,0.000000
29997,0.230869,4.015958,3.733505,2.862889,2.159589,-0.740329,0.763742,0.754710,0.048589,0.047318,0.039899,0.325221,0.339081,0.325219,0.000000,0.000000,1.251357,0.267328,0.131197,0.175565
29998,0.615650,4.450116,1.493402,-0.715722,0.719863,0.740329,0.763742,-0.754710,-0.022421,1.105113,1.103873,0.822071,0.195307,0.822313,5.184610,0.147645,0.067004,0.122589,3.474367,0.102168


In [6]:
#concatenating the encoded variables with the scaled variables
df_encoded_only = df_encoded.iloc[:,:9]
df_prep = pd.concat([df_encoded_only, df_scaled, df.def_next_month],axis=1)
df_prep

,SEX_1,SEX_2,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3,LIMIT_BAL,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,def_next_month
0,0,1,0,1,0,0,1,0,0,0.153912,2.604946,2.240103,2.147167,-0.719863,-0.740329,-0.763742,-0.754710,0.053332,0.043737,0.009968,0.000000,0.000000,0.000000,0.000000,0.029841,0.000000,0.000000,0.000000,0.000000,1
1,0,1,0,1,0,0,0,1,0,0.923475,2.822025,-0.746701,2.147167,0.719863,0.740329,0.763742,2.264129,0.036554,0.024322,0.038800,0.050969,0.056920,0.054788,0.000000,0.043310,0.056880,0.063649,0.000000,0.113268,1
2,0,1,0,1,0,0,0,1,0,0.692606,3.690340,0.746701,0.715722,0.719863,0.740329,0.763742,0.754710,0.398513,0.197775,0.196155,0.223237,0.246263,0.261240,0.091621,0.064965,0.056880,0.063649,0.065599,0.283170,0
3,0,1,0,1,0,0,1,0,0,0.384781,4.015958,0.746701,0.715722,0.719863,0.740329,0.763742,0.754710,0.640450,0.680066,0.713082,0.441053,0.477089,0.496422,0.120713,0.087444,0.068256,0.070014,0.070125,0.056634,0
4,1,0,0,1,0,0,1,0,0,0.384781,6.186746,-0.746701,0.715722,-0.719863,0.740329,0.763742,0.754710,0.117445,0.079945,0.518417,0.326187,0.315424,0.321422,0.120713,1.588665,0.568799,0.572845,0.045197,0.038454,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,1,0,0,0,1,0,1,0,0,1.693037,4.233037,0.746701,0.715722,0.719863,0.740329,0.763742,0.754710,2.575267,2.718615,3.014370,1.370856,0.514618,0.268482,0.513029,0.866206,0.284570,0.193940,0.327993,0.056634,0
29996,1,0,0,0,1,0,0,1,0,1.154343,4.667195,-0.746701,-0.715722,-0.719863,-0.740329,0.763742,0.754710,0.022938,0.025774,0.050663,0.139868,0.085503,0.000000,0.110875,0.152712,0.511805,0.008211,0.000000,0.000000,0
29997,1,0,0,1,0,0,0,1,0,0.230869,4.015958,3.733505,2.862889,2.159589,-0.740329,0.763742,0.754710,0.048589,0.047318,0.039899,0.325221,0.339081,0.325219,0.000000,0.000000,1.251357,0.267328,0.131197,0.175565,1
29998,1,0,0,0,1,0,1,0,0,0.615650,4.450116,1.493402,-0.715722,0.719863,0.740329,0.763742,-0.754710,-0.022421,1.105113,1.103873,0.822071,0.195307,0.822313,5.184610,0.147645,0.067004,0.122589,3.474367,0.102168,1


### PCA

In [7]:
# import PCA
from sklearn.decomposition import PCA

# instantiate PCA, fit on df_numeric_scaled, check first 10 entries of explained variance
pca = PCA()
pca.fit(df_prep)
pca.explained_variance_ratio_[:10]

array([0.30929795, 0.18374447, 0.0660845 , 0.05356694, 0.04275418,
       0.0410475 , 0.04022254, 0.03620429, 0.03379529, 0.03106701])

We see that there is no one linear combination of variables which explains a large part of the variance in the dataset. The first two components are the ones who explain the most, and they only reach 48% of total variance explained. This means that these findings may or may not be useful.

Although PCA is primarily a data-exploration method, it can also be used in Principal Components Regression (PCR), which we might explore in the modeling stage.

### Variable Importance

Now we want to get an idea of the importance (if any) of the numerical variables. The difficulty in doing this is that:

* running a full logistic regression on an unbalanced dataset will often not lead to convergence.
* including all variables might mask the importance of some variables, as the variance is explained by others.
* there might be multicollinearity between the variables.

We will therefore split the scaled numerical variables into a subset:

* all cash variables (LIMIT_BAL, BILL_AMT1 - BILL_AMT6, PAY_AMT1 - PAY_AMT6)

In [8]:
df_prep

,SEX_1,SEX_2,EDUCATION_1,EDUCATION_2,EDUCATION_3,EDUCATION_4,MARRIAGE_1,MARRIAGE_2,MARRIAGE_3,LIMIT_BAL,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,def_next_month
0,0,1,0,1,0,0,1,0,0,0.153912,2.604946,2.240103,2.147167,-0.719863,-0.740329,-0.763742,-0.754710,0.053332,0.043737,0.009968,0.000000,0.000000,0.000000,0.000000,0.029841,0.000000,0.000000,0.000000,0.000000,1
1,0,1,0,1,0,0,0,1,0,0.923475,2.822025,-0.746701,2.147167,0.719863,0.740329,0.763742,2.264129,0.036554,0.024322,0.038800,0.050969,0.056920,0.054788,0.000000,0.043310,0.056880,0.063649,0.000000,0.113268,1
2,0,1,0,1,0,0,0,1,0,0.692606,3.690340,0.746701,0.715722,0.719863,0.740329,0.763742,0.754710,0.398513,0.197775,0.196155,0.223237,0.246263,0.261240,0.091621,0.064965,0.056880,0.063649,0.065599,0.283170,0
3,0,1,0,1,0,0,1,0,0,0.384781,4.015958,0.746701,0.715722,0.719863,0.740329,0.763742,0.754710,0.640450,0.680066,0.713082,0.441053,0.477089,0.496422,0.120713,0.087444,0.068256,0.070014,0.070125,0.056634,0
4,1,0,0,1,0,0,1,0,0,0.384781,6.186746,-0.746701,0.715722,-0.719863,0.740329,0.763742,0.754710,0.117445,0.079945,0.518417,0.326187,0.315424,0.321422,0.120713,1.588665,0.568799,0.572845,0.045197,0.038454,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,1,0,0,0,1,0,1,0,0,1.693037,4.233037,0.746701,0.715722,0.719863,0.740329,0.763742,0.754710,2.575267,2.718615,3.014370,1.370856,0.514618,0.268482,0.513029,0.866206,0.284570,0.193940,0.327993,0.056634,0
29996,1,0,0,0,1,0,0,1,0,1.154343,4.667195,-0.746701,-0.715722,-0.719863,-0.740329,0.763742,0.754710,0.022938,0.025774,0.050663,0.139868,0.085503,0.000000,0.110875,0.152712,0.511805,0.008211,0.000000,0.000000,0
29997,1,0,0,1,0,0,0,1,0,0.230869,4.015958,3.733505,2.862889,2.159589,-0.740329,0.763742,0.754710,0.048589,0.047318,0.039899,0.325221,0.339081,0.325219,0.000000,0.000000,1.251357,0.267328,0.131197,0.175565,1
29998,1,0,0,0,1,0,1,0,0,0.615650,4.450116,1.493402,-0.715722,0.719863,0.740329,0.763742,-0.754710,-0.022421,1.105113,1.103873,0.822071,0.195307,0.822313,5.184610,0.147645,0.067004,0.122589,3.474367,0.102168,1


In [9]:
# We select only transaction variables
df_cashflow1 = df_prep.iloc[:,9]
df_cashflow2 = df_prep.iloc[:,11:]
df_cashflow = pd.concat([df_cashflow1,df_cashflow2],axis=1)
df_cashflow

,LIMIT_BAL,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,def_next_month
0,0.153912,2.240103,2.147167,-0.719863,-0.740329,-0.763742,-0.754710,0.053332,0.043737,0.009968,0.000000,0.000000,0.000000,0.000000,0.029841,0.000000,0.000000,0.000000,0.000000,1
1,0.923475,-0.746701,2.147167,0.719863,0.740329,0.763742,2.264129,0.036554,0.024322,0.038800,0.050969,0.056920,0.054788,0.000000,0.043310,0.056880,0.063649,0.000000,0.113268,1
2,0.692606,0.746701,0.715722,0.719863,0.740329,0.763742,0.754710,0.398513,0.197775,0.196155,0.223237,0.246263,0.261240,0.091621,0.064965,0.056880,0.063649,0.065599,0.283170,0
3,0.384781,0.746701,0.715722,0.719863,0.740329,0.763742,0.754710,0.640450,0.680066,0.713082,0.441053,0.477089,0.496422,0.120713,0.087444,0.068256,0.070014,0.070125,0.056634,0
4,0.384781,-0.746701,0.715722,-0.719863,0.740329,0.763742,0.754710,0.117445,0.079945,0.518417,0.326187,0.315424,0.321422,0.120713,1.588665,0.568799,0.572845,0.045197,0.038454,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,1.693037,0.746701,0.715722,0.719863,0.740329,0.763742,0.754710,2.575267,2.718615,3.014370,1.370856,0.514618,0.268482,0.513029,0.866206,0.284570,0.193940,0.327993,0.056634,0
29996,1.154343,-0.746701,-0.715722,-0.719863,-0.740329,0.763742,0.754710,0.022938,0.025774,0.050663,0.139868,0.085503,0.000000,0.110875,0.152712,0.511805,0.008211,0.000000,0.000000,0
29997,0.230869,3.733505,2.862889,2.159589,-0.740329,0.763742,0.754710,0.048589,0.047318,0.039899,0.325221,0.339081,0.325219,0.000000,0.000000,1.251357,0.267328,0.131197,0.175565,1
29998,0.615650,1.493402,-0.715722,0.719863,0.740329,0.763742,-0.754710,-0.022421,1.105113,1.103873,0.822071,0.195307,0.822313,5.184610,0.147645,0.067004,0.122589,3.474367,0.102168,1


In [23]:
import statsmodels.api as sm

# Log Reg model for cashflow and summary
cashflow_model = sm.Logit(df_prep.def_next_month, df_cashflow).fit()
cashflow_model.summary()

Optimization terminated successfully.
         Current function value: 0.517690
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:         def_next_month   No. Observations:                29601
Model:                          Logit   Df Residuals:                    29582
Method:                           MLE   Df Model:                           18
Date:                Fri, 04 Mar 2022   Pseudo R-squ.:                 0.02478
Time:                        17:24:54   Log-Likelihood:                -15324.
converged:                       True   LL-Null:                       -15714.
Covariance Type:            nonrobust   LLR p-value:                9.723e-154
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
LIMIT_BAL     -0.7099      0.017    -41.891      0.000      -0.743      -0.677
PAY_1          0.2600      0.018     14.243      0.000       0.224       0.296
PAY_2          0.0192      0.024      0.789      0.430      -0.028       0.067
PAY_3         -0.0102      0.026     -0.388      0.698      -0.061       0.041
PAY_4         -0.0067      0.028     -0.243      0.808      -0.061       0.048
PAY_5          0.0066      0.029      0.232      0.817      -0.049       0.062
PAY_6         -0.0579      0.024     -2.379      0.017      -0.106      -0.010
BILL_AMT1     -0.9357      0.103     -9.083      0.000      -1.138      -0.734
BILL_AMT2      0.4119      0.127      3.249      0.001       0.163       0.660
BILL_AMT3      0.1214      0.110      1.108      0.268      -0.093       0.336
BILL_AMT4      0.0646      0.103      0.630      0.529      -0.136       0.266
BILL_AMT5      0.3117      0.106      2.935      0.003       0.104       0.520
BILL_AMT6      0.0793      0.082      0.968      0.333      -0.081       0.240
PAY_AMT1      -0.6321      0.057    -11.066      0.000      -0.744      -0.520
PAY_AMT2      -0.5854      0.071     -8.284      0.000      -0.724      -0.447
PAY_AMT3      -0.1870      0.040     -4.693      0.000      -0.265      -0.109
PAY_AMT4      -0.1731      0.035     -4.926      0.000      -0.242      -0.104
PAY_AMT5      -0.1067      0.034     -3.159      0.002      -0.173      -0.040
PAY_AMT6      -0.0314      0.026     -1.190      0.234      -0.083       0.020
==============================================================================
"""

We see that there are more variables that are potentially important. They are 11 (out of 19) and they are: LIMIT_BAL, PAY_1, PAY_6, BILL_AMT1, BILL_AMT2, BILL_AMT5, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5.

All coefficients are small apart from BILL_AMT1 and LIMIT_BAL- meaning these could be important variables for modeling.

#### Conclusions about importance

Although many variables seem to show importance when predicting Default or No Default, not all of these variables may equally contribute to future models. The idea of this exploration is just to get an idea of what variables we might want to experiment with in our models.